## Please run the code cell below to correctly initialize the notebook

If a bug is found an issue can be created on the [Github](https://github.com/saullocastro/tuduam/issues) page

In [ ]:
!pip install tuduam
!git clone https://github.com/saullocastro/tuduam.git
# If the following error occurs "fatal: destination path 'tuduam' already exists and is not an empty directory." please continue as your environment is already set up correctly

# Structural analysis of a wing section

Performing a stress analysis of the complex geometry of the multicell wingbox shown in [figure 1](#figure1) can be complicated and tedious. Additionally, for a preliminary investigation often speed and simplicity are of greater importance, on the other hand a final solution must be as exact as circumstances allow. In order to meet these standard we idealize the wingbox into a simpler mechanical model which under given loading conditions behaves in nearly the same manner as the original structure.

<br>
<p align="left"  id="figure1" >
<img src="https://github.com/saullocastro/tuduam/blob/main/notebooks/static/idealization_wingbox_corr.png?raw=true" width="800" /> 
 <br>
</p>
<p align="left">
<em>Figure 1: The NACA 4412 airfoil</em>
</p>
<br>

To get to this mechical model we replace the stringers and spar flanges with concentrations of areas called booms over which the direct stress is constant and which are located along the midline of the skin. Furthermore we assume that all shear stresses are carried by the skin and all direct stresses by these booms. An important consideration here is that the direct stress carrying capacity of the skin may be allowed for by increasing each boom area by an area equivalent to the direct stress carrying capacity of the adjacent skin panels. Using this assumption the shearflow through the skin section between the booms will be constant and will only change when encountering a boom. If you want more details on this idealization please refer to source 1. 

## The skin addition to the boom areas

As previously mentioned in order to make the assumption that the skin thickness is zero we first have to incorportate the direct stress carrying capacity of the skin into the booms. To find this addition we can perform a moment equivalence for the two panels shown [figure 2](#figure2).  This wil result in the equation 1.

<br>

$$
\begin{equation}
\tag{1}
B_2 = \frac{t_D b}{6}(2 + \frac{\sigma_1}{\sigma_2})
\end{equation}
$$

Where $\sigma_1$, $\sigma_2$ are the direct stresses at boom 1 and 2, respectively. As seen from equation 1, the idealization is dependent on the loading case as the direct stresses would change for different loadings. However, you might correctly see the chicken and egg problem here. Therefore, we have to rewrite equation 1, we can do this by assuming that the direct stress is mostly related to the lift and therefore using $\sigma = \frac{M \cdot y}{Ixx}$ we can rewrite equation 1 to equation 2.

$$
\begin{equation}
\tag{2}
B_2 = \frac{t_D b}{6}(2 + \frac{y_1}{y_2})
\end{equation}
$$
 
<br>
<p align="left"  id="figure2" >
<img src="https://github.com/saullocastro/tuduam/blob/main/notebooks/static/idealization_panel.png?raw=true" width="500" /> 
 <br>
</p>
<p align="left">
<em>Figure 1: The NACA 4412 airfoil</em>
</p>
<br>

## Sources

1. Megson T. H. G. 1972. Aircraft Structures for Engineering Students. London: Edward Arnold. 

In [ ]:
import tuduam as tud
import os 
import numpy as np


data_path = os.path.join(".",'tuduam/tests/naca45112.txt')
chord = 2 # meters
attr_dict = {
    "n_cell":4,
    "spar_loc_nondim":[0.3, 0.5, 0.7],
    "t_sk_cell":[0.002,0.004,0.004, 0.004],
    "area_str":20e-6,
    "t_sp":0.008,
    "str_cell":[8,8,7,8],
    "booms_sk": 27,
    "booms_spar": 5,
}


# 
wingbox_struct = tud.Wingbox(**attr_dict)
wingbox = tud.discretize_airfoil(data_path, chord, wingbox_struct)
wingbox.plot_geometry()

# Print the properties of the booms
print("------------- Properties Table --------------------")
print("id - X loc - Y loc --- Boom area")
for i in wingbox.boom_dict.values():
  print(f"{i.bid} - {np.round(i.x,2)} [m] - {np.round(i.y,2)} [m] - {np.round(i.A*1e6)} [mm^2]")

# Direct Stress Analysis

Direct stress in the cross section of a wingbox are caused by the bending moments caused by the lift and drag. However because the lift forces are significantly larger than the drag forces involved we have chosen to for now ignore the stresses caused by the drag in our model. Additionally, we can generally say the contribution $I_{xy}$ of neglible hence it also left out. In future works this could also be taken into account however. Using these assume leaves us with equation 3.

$$
\begin{equation}
\tag{3}
\sigma_z = \frac{M_y \cdot y}{I_{xx}}
\end{equation}
$$

Getting the values of $I_{xx}$ is very much so simplified due to the definition of our booms. It can simply be found by using equation 4.

$$
\begin{equation}
\tag{4}
I_{xx} = \sum^n_{n =0} B_n y_n^2 
\end{equation}
$$

It is then straightforward to get the stress at each boom. See the code cell below.

In [ ]:
import rich

#-------temporary bugfix please ignore-------------
for pnl in wingbox.panel_dict.values():
  pnl.q_basic = None
  pnl.q_total = None
#--------------------------------------------------


wingbox.stress_analysis(4e3, 38e3, 0.3, 24e9)
wingbox.plot_direct_stresses()

print("\n \n")
rich.print(f"Maximum stress is {max([i.sigma for i in wingbox.boom_dict.values()])/1e6} MPa")
rich.print(f"$Ixx = {wingbox.Ixx*1e12} mm^4")

# Shear Stress analysis

The direct stresses were straight forward however computing the shear stresses is quite more involved. As mentioned earlier, the shear flow over a panel is constant and only jumps at the one of the booms. When we have an open sections this gives us an easy point to start from as we know that at the outer edge of the structure the shear flow has to be zero. However, a wingbox is a closed strucutre so we have to cut the structure in each cell to give us a starting point. This is as shown in [figure 3](#figure3) indicated by the two small parrallel lines. 

<br>
<p align="left"  id="figure3" >
<img src="https://github.com/saullocastro/tuduam/blob/main/notebooks/static/wingbox_analysis.png?raw=true" width="800" /> 
 <br>
</p>
<p align="left">
<em>Figure 3: The NACA 4412 airfoil</em>
</p>
<br>

However, these cut's have to be "mended", we do this by applying a complementary shear flow in each cell. We can solve for these complementary shear flows by assuming no warping will occur and hence the amount of twist per unit length should be the same in each cell. This would give us n + 1 unknowns however with n equations. Thus, the final equation can be found from moment equivalence of all the shear flows and the internal shear itself. A complete description of this process is described in source 1 which has been previously shown. Now let us apply it. See the code cell below.

In [ ]:
import rich
import plotly.graph_objs as go
#-------temporary bugfix please ignore-------------
for pnl in wingbox.panel_dict.values():
  pnl.q_basic = None
  pnl.q_total = None
#--------------------------------------------------


wingbox.stress_analysis(4e3, 38e3, 0.3, 24e9) 
wingbox.plot_shear_stress()



y_arr  = [i.y for i in wingbox.boom_dict.values()]
y_max = np.max(y_arr)
y_min = np.min(y_arr)

# Sample data and color mapping - replace with your actual data
stress_values = np.abs([panel.tau for panel in wingbox.panel_dict.values()])/1e6
norm = (stress_values - stress_values.min()) / (stress_values.max() - stress_values.min())

# Create traces
traces = []
for key, panel in wingbox.panel_dict.items():
    x = [panel.b1.x, panel.b2.x]
    y = [panel.b1.y, panel.b2.y]
    stress = abs(panel.tau/1e6)
    normalized_stress = (stress - stress_values.min()) / (stress_values.max() - stress_values.min())
    color = f"rgb({normalized_stress*255}, {(1-normalized_stress)*255}, 0)"  # Color from blue to red based on stress

    trace = go.Scatter(
        x=x,
        y=y,
        mode='lines+markers',
        marker=dict(color=color, symbol='triangle-right'),
        line=dict(color=color),
        showlegend=False
    )
    traces.append(trace)

# Layout configuration
layout = go.Layout(
    title='Panel Stress Visualization',
    xaxis=dict(title='X-axis'),
    yaxis=dict(title='Y-axis', range=[y_min - 0.1, y_max + 0.1]),
    coloraxis=dict(colorscale='Viridis', colorbar=dict(title='Stress Value')),
)

# Create figure and add traces
fig = go.Figure(data=traces, layout=layout)

# Show plot
fig.show()


print("\n \n")
rich.print(f"Maximum shear stress is {max([i.tau for i in wingbox.panel_dict.values()])/1e6} MPa")